In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [9]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [67]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)




emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [68]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [69]:
C = torch.randn((27, 2))
C

tensor([[-1.0446e+00, -1.2629e+00],
        [ 1.6883e-01,  2.1109e-01],
        [ 3.7803e-01,  1.8099e+00],
        [ 2.2987e-01,  1.2984e+00],
        [ 1.8338e-01,  2.6102e-01],
        [ 6.2484e-01, -3.9319e-05],
        [-3.4088e+00,  7.6654e-01],
        [-1.0030e+00, -1.2996e+00],
        [ 7.1890e-01, -3.9310e-01],
        [ 3.6166e-01,  7.0639e-01],
        [ 9.7822e-02, -8.4295e-01],
        [-1.8001e-01,  9.7387e-01],
        [ 7.8929e-02, -7.0273e-01],
        [-8.8248e-02, -5.6384e-01],
        [-6.4961e-01,  8.7236e-01],
        [ 1.1868e-02,  1.1953e+00],
        [-7.1521e-01, -6.2805e-01],
        [-2.7755e-01, -1.0165e+00],
        [-1.7124e+00,  1.3979e+00],
        [ 4.9428e-01, -1.9265e-01],
        [ 8.6645e-01,  2.7560e-01],
        [ 4.2611e-01,  4.4516e-01],
        [-7.8031e-01,  3.6138e-01],
        [-2.5396e-01,  7.3010e-01],
        [ 7.4878e-01,  8.2548e-01],
        [ 3.1551e-01,  2.7884e+00],
        [-3.3213e-01, -2.0148e+00]])

In [70]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [71]:
sum(p.nelement() for p in parameters)

11897

In [72]:
for p in parameters:
    p.requires_grad = True

In [75]:

for _ in range(30):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    # update parameters
    for p in parameters:
        p.data += -0.1 * p.grad



2.489231824874878
1.9549765586853027
1.6265922784805298
1.36825692653656
1.079801082611084
0.9312989711761475
0.8307801485061646
0.7294750809669495
0.45205146074295044
0.3853428363800049
0.37679582834243774
0.3333559036254883
0.3128680884838104
0.3077017366886139
0.31360068917274475
0.29718610644340515
0.2918201982975006
0.288479745388031
0.2879208028316498
0.2834346294403076
0.2820073366165161
0.27919867634773254
0.2780517339706421
0.2759515643119812
0.2749682664871216
0.2733875811100006
0.2725551128387451
0.2713235914707184
0.27061766386032104
0.2696288228034973
